In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


You will need to setup git, adapt your email and name in the following cell.

In [26]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from transformers import TFAutoModelForMaskedLM, AutoModelForMaskedLM

model_checkpoint = "google/electra-base-generator"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/231M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-base-generator were not used when initializing TFElectraForMaskedLM: ['activation']
- This IS expected if you are initializing TFElectraForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraForMaskedLM were initialized from the model checkpoint at google/electra-base-generator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraForMaskedLM for predictions without further training.


In [3]:
model.summary()

Model: "tf_electra_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 electra (TFElectraMainLaye  multiple                  33511168  
 r)                                                              
                                                                 
 generator_predictions (TFE  multiple                  198912    
 lectraGeneratorPredictions                                      
 )                                                               
                                                                 
 generator_lm_head (TFElect  multiple                  23867706  
 raMaskedLMHead)                                                 
                                                                 
Total params: 33740602 (128.71 MB)
Trainable params: 33740602 (128.71 MB)
Non-trainable params: 0 (0.00 Byte)
________________________________________________________________

In [4]:
text = "She works as a [MASK]."

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> She works as a nurse.
>>> She works as a teacher.
>>> She works as a consultant.
>>> She works as a nanny.
>>> She works as a waitress.


In [7]:
from datasets import load_dataset

stories_dataset = load_dataset("deven367/babylm-100M-children-stories")
stories_dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/76758 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/5996 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7959 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 76758
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 5996
    })
    test: Dataset({
        features: ['text'],
        num_rows: 7959
    })
})

In [8]:
sample = stories_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['text']}'")


'>>> Review: Then her father got into a mighty rage and swore a great big oath, and said: “To-morrow, so sure as I live and eat, I'll twist that birdie's neck,” and out he stamped from her room.'

'>>> Review: In the first place she told the maiden to go out of the den and to seek the cave of the magician, U Hynroh, the Giant Toad, to whom the realm was under tribute. He was a peevish and exacting monster from whom every one recoiled, and Ka Nam would have been terrified to approach him under ordinary conditions, but the peril which faced her gave her courage, and under the guidance of the mouse she went to the toad's cave. When he saw her and beheld how fair she was, and learned how she had been the captive of his old rival the tiger, he readily consented to give her his protection; so he clothed her in a toadskin, warning her not to divest herself of it in the presence of others on pain of death. This he did in order to keep the maiden in his own custody and to make her his slave.'


In [9]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_datasets = stories_dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

Map:   0%|          | 0/76758 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/5996 [00:00<?, ? examples/s]

Map:   0%|          | 0/7959 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 76758
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 5996
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 7959
    })
})

In [10]:
tokenizer.model_max_length

512

In [11]:
chunk_size = 128

In [12]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 6'
'>>> Review 1 length: 53'
'>>> Review 2 length: 65'


In [13]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 124'


In [14]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 124'


In [15]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [16]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/76758 [00:00<?, ? examples/s]

Map:   0%|          | 0/5996 [00:00<?, ? examples/s]

Map:   0%|          | 0/7959 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 33596
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2709
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3380
    })
})

In [17]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'’ t you be like the happy prince? ” asked a sensible mother of her little boy who was crying for the moon. “ the happy prince never dreams of crying for anything. ” [SEP] [CLS] “ i am glad there is some one in the world who is quite happy, ” muttered a disappointed man as he gazed at the wonderful statue. [SEP] [CLS] “ he looks just like an angel, ” said the charity children as they came out of the cathedral in their bright scarlet cloaks and their clean white pinafores. [SEP] [CLS] “ how do you know? ” said the mathematical master, “ you have never seen one. ” [SEP] [CLS]'

In [18]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [19]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a ElectraTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] the happy prince. [SEP] [CLS] high above the city, on a tall column, stood the statue of [MASK] happyoured [MASK] he was gilded all [MASK] with thin leaves of fine gold, for eyes he [MASK] two bright sapphires, and a large red ruby glowed on his sword [MASK] hilt. [SEP] [CLS] [MASK] [MASK] very much admired [MASK]. “ he is as beautiful as a weathercock, ” remarked one of the town councillors who wished [MASK] gain a reputation for having artistic tastes ; “ only not quite so useful, ” he added, fearing lest people should think him unpr [MASK]al, which he really was not. [SEP] [CLS] [MASK] why can'

'>>> [MASK] t you be [MASK] the happy prince [MASK] ” asked a sensible mother of her little [MASK] [MASK] was crying for the moon. “ the happy prince never dreams of crying for anything. ” [SEP] [CLS] “ i am glad there is some one in the world who is quite happy [MASK] ” muttered [MASK] disappointed man as he gazed at the wonderful statue. [SEP] [CLS] “ [MASK] looks just like an 

In [20]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [21]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] the happy prince. [SEP] [CLS] high above [MASK] [MASK], on a [MASK] [MASK], stood the statue of the happy prince [MASK] he was [MASK] all over with thin [MASK] of fine gold, for eyes he had two [MASK] sapphires, and [MASK] large red ruby [MASK] on his sword - hilt. [SEP] [CLS] he was very much admired indeed. [MASK] he is as beautiful as a weathercock, ” remarked one [MASK] the town councillors who wished to gain a [MASK] for having [MASK] tastes ; “ only not quite so useful, ” he [MASK], [MASK] lest people should think him unpractical, [MASK] he really was not. [SEP] [CLS] “ [MASK] can'

'>>> ’ [MASK] you be like the happy [MASK]? ” asked a sensible [MASK] of her little boy who was crying for the moon. “ the [MASK] prince never dreams [MASK] crying for anything [MASK] ” [SEP] [CLS] “ i am glad there is some one in [MASK] world who [MASK] quite happy, ” muttered a disappointed man [MASK] he gazed at the [MASK] statue. [SEP] [CLS] “ he looks just like [MASK] angel, ” said th

In [22]:
train_size = 10_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [24]:
tf_train_dataset = model.prepare_tf_dataset(
    downsampled_dataset["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = model.prepare_tf_dataset(
    downsampled_dataset["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

In [27]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-stories", tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/viditnaik/electra-base-generator-finetuned-stories into local empty directory.


In [28]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 12s 145ms/step - loss: 5.0928
Perplexity: 162.84


In [29]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback])

312/312 [==============================] - 142s 412ms/step - loss: 3.8062 - val_loss: 2.8854


In [30]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 5s 147ms/step - loss: 2.9084
Perplexity: 18.33
